In [29]:
import serial
serial_port = "/dev/ttyUSB0"
baudrate = 115200
serial = serial.Serial(serial_port, baudrate)

In [28]:
serial.close()

In [206]:
#GRBL view settings?
serial.write(b"\n\r")
time.sleep(1.0)
res = serial.read_all()
print(res)

b'echo:Unknown command: "?"\nok\n'


In [62]:
serial.write(b"G90\n\r") # Set to absolute mode
serial.write(b"G21\n\r") # Set to mm

5

In [63]:
# Pen Down
serial.write(b"M280P0S0\n\r")

10

In [18]:
# Pen Up (HIGH)

serial.write(b"M280P0S90\n\r")

11

In [65]:
# Pen Up (Normal)

serial.write(b"M280P0S30\n\r")

11

In [57]:
# def pen_to_angle(serial, angle):
#     a = b"M28P0S{int(angle)}".encode("ascii")
#     serial.write(a)
#     serial.write(b"\n\r")
# pen_to_angle(serial, 0)   

In [79]:
# Pause 
serial.write(b"M280P0S0\n\r") # pen down
serial.write(b"G4 P2000\n\r") # P50 is the pause length in ms
serial.write(b"M280P0S30\n\r") # pen up


11

In [127]:
# Auto home
serial.write(b"G28\n\r")

5

In [76]:
# Go to position
x = 100
y = 120.0

max_y = 300 #mm
max_x = 300 #mm
sendstr = f"G1 X{x:0.3f} Y{y:0.3f}\n\r".encode('utf-8')
print(sendstr)
serial.write(sendstr)

b'G1 X100.000 Y120.000\n\r'


22

29

In [141]:
# Travel speed
serial.write(b"G1 F10000\n\r")

11

In [143]:
#Paint Speed
# Travel speed
serial.write(b"G1 F3000\n\r")

10

In [98]:
import enum
from serial import Serial
from typing import Tuple, List, Optional

Point = Tuple[float, float]

class PenState(enum.Enum):
    DOWN = 0
    UP = 30
    HIGH_UP = 90

class Rotation(enum.Enum):
    CLOCKWISE = 2
    COUNTER_CLOCKWISE = 3

class CommandException(Exception): pass

class SovolSO1(object):
    """
    Object for controlling the Sovol SO1 pen plotter given the G-Code here:

    https://marlinfw.org/docs/gcode/G005.html
    """
    def __init__(self, port="/dev/ttyUSB1", baud=115200, max_dimensions=(300.0, 300.0), timeout=1.0):
        self.serial = Serial(port, baud, dsrdtr=True, timeout=.0005)
        self.end_of_line = b"\n"
        self.max_dimensions = max_dimensions

        # Wait until the machine has started up, after some amount of time
        # establishing a new connection will cause the machine to reboot.
        # It would be good to figure out how to make this a little more robust
        time.sleep(10.0)
        startup = self.serial.read_all()
        print(startup.decode('ascii'))
        self.write("G90") # Set to absolute mode
        self.write("G21") # Set to mm
        

    def close(self):
        self.serial.close()

    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.disableSteppers()
        self.close()

    def disableSteppers(self, timeout: Optional[float] = None, send_x_flag = False, send_y_flag=False, send_z_flag=False):
        command = "M18"
        if timeout: 
            command += " S{timeout:0.0f}"
        if send_x_flag:
            command += " X"
        if send_y_flag:
            command += " Y"
        if send_z_flag:
            command += " Z"
        self.write(command)

    def enableSteppers(self, send_x_flag = False, send_y_flag=False, send_z_flag=False):
        command = "M17"
        if send_x_flag:
            command += " X"
        if send_y_flag:
            command += " Y"
        if send_z_flag:
            command += " Z"
        self.write(command)

    def write(self, cmd: str, timeout = 100.0):
        success = False

        # self.serial.flush() # Make sure we don't have any garbage left over.
        
        print(cmd)
        #Write the command
        self.serial.write(cmd.encode('utf-8') + self.end_of_line)

        ## Wait until we get an ok returned
        start_time = time.time()
        read_buffer = ""
        while (not success) and ((time.time() - start_time) < timeout):
            # To do this properly we really should have a buffer incase we wrap over. 
            read_buffer += self.serial.read().decode('ascii')
            if "ok" in read_buffer:
                print(read_buffer)
                print( "Success!!!")
                success = True
        if not success:
            print(read_buffer)
            raise CommandException("Failed to get a successful command")            

    def setPen(self, state: PenState):
        self.write(f"M280P0S{state.value}")

    def pause(self, pause_time_ms: int):
        self.write(f"G4 P{pause_time_ms}")

    def moveTo(self, point: Point):
        self.write(f"G1 X{point[0]:0.3f} Y{point[1]:0.3f}")

    def arcTo(self, point: Optional[Point] = None, center: Optional[Point] = None, rot: Rotation = Rotation.CLOCKWISE, radius: Optional[float] = None):
        if center:
            if point:
                self.write(
                    f"G{rot.value} "
                    f"X{point[0]:0.3f} Y{point[1]:0.3f} "
                    f"I{center[0]:0.3f} J{center[1]:0.3f}")
            else:
                self.write(
                    f"G{rot.value} "
                    f"I{center[0]:0.3f} J{center[1]:0.3f}")

        elif radius:
            self.write(
                f"G{rot.value} "
                f"X{point[0]:0.3f} Y{point[1]:0.3f} "
                f"R{radius:0.3f}")
        else:
            raise CommandException("Must provide either a center point or a radius")


    def autoHome(self):
        self.write("G28", timeout=100.0)

    def setSpeed(self, speed: int):
        self.write(f"G1 F{speed}")

    def setTravelSpeed(self):
        self.setSpeed(10000)
    
    def setDrawingSpeed(self):
        self.setSpeed(3000)

In [7]:
import time
with SovolSO1("/dev/ttyUSB0") as plotter:
    plotter.autoHome()
    plotter.setTravelSpeed()
    plotter.setPen(PenState.HIGH_UP)
    plotter.pause(50)
    plotter.moveTo((75.0,75.0))
    plotter.setDrawingSpeed()
    plotter.setPen(PenState.DOWN)
    plotter.pause(50)
    plotter.moveTo((100.0,100.0))
    plotter.moveTo((50.0,100.0))
    plotter.moveTo((75.0,75.0))
    plotter.setPen(PenState.HIGH_UP)
    plotter.pause(50)
    plotter.moveTo((50.0,100.0))
    plotter.setPen(PenState.DOWN)
    plotter.pause(50)
    plotter.moveTo((50.0,50.0))
    plotter.moveTo((75.0,75.0))
    plotter.setPen(PenState.HIGH_UP)
    plotter.pause(50)
    plotter.moveTo((50.0,50.0))
    plotter.setPen(PenState.DOWN)
    plotter.pause(50)
    plotter.moveTo((100.0,50.0))
    plotter.moveTo((75.0,75.0))
    plotter.setPen(PenState.HIGH_UP)
    plotter.pause(50)
    plotter.moveTo((100.0,50.0))
    plotter.setPen(PenState.DOWN)
    plotter.pause(50)
    plotter.moveTo((100.0,100.0))
    plotter.arcTo((50.0,100.0), (-25.0,-25.0), Rotation.COUNTER_CLOCKWISE)
    plotter.arcTo((50.0,50.0), (25.0,-25.0), Rotation.COUNTER_CLOCKWISE)
    plotter.arcTo((100.0,50.0), (25.0,25.0), Rotation.COUNTER_CLOCKWISE)
    plotter.arcTo((100.0,100.0), (-25.0,25.0), Rotation.COUNTER_CLOCKWISE)
    plotter.setPen(PenState.HIGH_UP)
    plotter.pause(50)
    plotter.setTravelSpeed()
    plotter.moveTo((0.0,0.0))

    

start
echo: External Reset
Marlin 1.1.6

echo: Last Updated: 2019-12-16 | Author: SO-1
echo:Compiled: Mar 16 2020
echo: Free Memory: 10510  PlannerBufferBytes: 1232
echo:EEPROM version mismatch (EEPROM=? Marlin=V41)
echo:Hardcoded Default Settings Loaded
echo:  G21    ; Units in mm
echo:  M149 C ; Units in Celsius

echo:Filament settings: Disabled
echo:  M200 D1.75
echo:  M200 D0
echo:Steps per unit:
echo:  M92 X80.00 Y80.00 Z400.00 E140.00
echo:Maximum feedrates (units/s):
echo:  M203 X300.00 Y300.00 Z5.00 E25.00
echo:Maximum Acceleration (units/s2):
echo:  M201 X1000 Y1000 Z100 E1000
echo:Acceleration (units/s2): P<print_accel> R<retract_accel> T<travel_accel>
echo:  M204 P1000.00 R1000.00 T1000.00
echo:Advanced: S<min_feedrate> T<min_travel_feedrate> B<min_segment_time_ms> X<max_xy_jerk> Z<max_z_jerk> E<max_e_jerk>
echo:  M205 S0.00 T0.00 B20000 X10.00 Y10.00 Z0.40 E5.00
echo:Home offset:
echo:  M206 X0.00 Y0.00 Z0.00
echo:Material heatup parameters:
echo:  M145 S0 H185 B45 F0
  M14

In [300]:
with SovolSO1("/dev/ttyUSB3") as plotter:
    plotter.setPen(PenState.DOWN)
    time.sleep(10.0)

start
echo: External Reset
Marlin 1.1.6

echo: Last Updated: 2019-12-16 | Author: SO-1
echo:Compiled: Mar 16 2020
echo: Free Memory: 10510  PlannerBufferBytes: 1232
echo:EEPROM version mismatch (EEPROM=? Marlin=V41)
echo:Hardcoded Default Settings Loaded
echo:  G21    ; Units in mm
echo:  M149 C ; Units in Celsius

echo:Filament settings: Disabled
echo:  M200 D1.75
echo:  M200 D0
echo:Steps per unit:
echo:  M92 X80.00 Y80.00 Z400.00 E140.00
echo:Maximum feedrates (units/s):
echo:  M203 X300.00 Y300.00 Z5.00 E25.00
echo:Maximum Acceleration (units/s2):
echo:  M201 X1000 Y1000 Z100 E1000
echo:Acceleration (units/s2): P<print_accel> R<retract_accel> T<travel_accel>
echo:  M204 P1000.00 R1000.00 T1000.00
echo:Advanced: S<min_feedrate> T<min_travel_feedrate> B<min_segment_time_ms> X<max_xy_jerk> Z<max_z_jerk> E<max_e_jerk>
echo:  M205 S0.00 T0.00 B20000 X10.00 Y10.00 Z0.40 E5.00
echo:Home offset:
echo:  M206 X0.00 Y0.00 Z0.00
echo:Material heatup parameters:
echo:  M145 S0 H185 B45 F0
  M14

In [101]:
import time
import random
import numpy

with SovolSO1("/dev/ttyUSB1") as plotter:
    # plotter.autoHome()
    plotter.setTravelSpeed()
    plotter.setPen(PenState.HIGH_UP)
    plotter.pause(50)
    plotter.moveTo((100.0,150.0))
    plotter.pause(50)
    plotter.setDrawingSpeed()
    plotter.pause(1000)
    plotter.setPen(PenState.DOWN)
    plotter.pause(50)

    for i in range(1):
        center = (random.randrange(-20,20), random.randrange(-20,20))
        if center[0] < 0:
            rot = Rotation.COUNTER_CLOCKWISE
        else:
            rot = Rotation.CLOCKWISE

        plotter.arcTo(center=center, rot=rot)
 
    plotter.setPen(PenState.HIGH_UP)
    plotter.pause(50)   
    plotter.setTravelSpeed()
    plotter.moveTo((0.0,0.0))

    

start
echo: External Reset
Marlin 1.1.6

echo: Last Updated: 2019-12-16 | Author: SO-1
echo:Compiled: Mar 16 2020
echo: Free Memory: 10510  PlannerBufferBytes: 1232
echo:EEPROM version mismatch (EEPROM=? Marlin=V41)
echo:Hardcoded Default Settings Loaded
echo:  G21    ; Units in mm
echo:  M149 C ; Units in Celsius

echo:Filament settings: Disabled
echo:  M200 D1.75
echo:  M200 D0
echo:Steps per unit:
echo:  M92 X80.00 Y80.00 Z400.00 E140.00
echo:Maximum feedrates (units/s):
echo:  M203 X300.00 Y300.00 Z5.00 E25.00
echo:Maximum Acceleration (units/s2):
echo:  M201 X1000 Y1000 Z100 E1000
echo:Acceleration (units/s2): P<print_accel> R<retract_accel> T<travel_accel>
echo:  M204 P1000.00 R1000.00 T1000.00
echo:Advanced: S<min_feedrate> T<min_travel_feedrate> B<min_segment_time_ms> X<max_xy_jerk> Z<max_z_jerk> E<max_e_jerk>
echo:  M205 S0.00 T0.00 B20000 X10.00 Y10.00 Z0.40 E5.00
echo:Home offset:
echo:  M206 X0.00 Y0.00 Z0.00
echo:Material heatup parameters:
echo:  M145 S0 H185 B45 F0
  M14